In [4]:
import pandas as pd
import numpy as np
import requests
import sys

In [5]:
klist="./mock_keyword_list.txt"
glist="./example_list.txt"

In [ ]:
def append_summary():
    with open(klist, "r") as fk:
        pheno=[]
        for line in fk:
            pheno.append(line.rstrip())
        #print(pheno)
        summary=pd.read_csv(glist, sep="\t", header=None)
        #summary.columns=["GENE"]
        genes=list(summary[0])
        return genes

        #creating knetminer genepage urls.
        network_view=[]
        keyw2 = "+OR+".join("({})".format(i.replace(" ", "+AND+")) for i in pheno)
        #print(keyw2)
        #define species
        species="wheatknet"
        for i in genes:
            link="http://knetminer.rothamsted.ac.uk/{}/genepage?list={}&keyword={}".format(species, i, keyw2)
            r=requests.get(link)
            network_view.append(r.url)

        #obtaining knetscores for genes
        keyw1 = "%20OR%20".join("({})".format(i.replace(" ", "+AND+")) for i in pheno)
        genestr=(",").join(genes)
        link="http://knetminer.rothamsted.ac.uk/{}/genome?".format(species)
        parameters={"keyword":keyw1, "list":genestr}
        r=requests.get(link, params=parameters)

        #check if requests is successful
        if not r.ok:
                r.raise_for_status()
                sys.exit()
        
        #extract unicode string of geneTable decoded from json
        decoded=r.json()[u'geneTable'].split("\t")
        #remove space or newline at the end
        decoded=(decoded)[:-1]
        
        colnum=9
        #tabulate genetable into 9 columns.
        genetable=np.array(decoded).reshape(len(decoded)//colnum, colnum)
        genetable=pd.DataFrame(genetable[1:,:], columns=genetable[0,:])
                
        knetgenes=list(genetable[u'ACCESSION'])
        knetscores=list(genetable[u'SCORE'])
        knetchro=list(genetable[u'CHRO'])
        knetstart=list(genetable[u'START'])

        #map genes to snps via a dictionary.
        knetdict=dict(zip(knetgenes, knetscores))
        ordered_score=[]
        for i in summary[0]:
            #convert gene id to upper case to avoid sensitivity issues.
            i=i.upper()
            ordered_score.append(knetdict[u'{}'.format(i)])
        summary[u'knetscore'] = ordered_score
        summary[u'network_view']=network_view

        summary

In [ ]:
append_summary()